In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename

In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
               or path.count('Fraud') 
               or path.count('Product')
              ]
test_paths = [path for path in test_paths 
               if path.count('DT') 
               or path.count('Fraud') 
               or path.count('Product')
              ]

df_train = parallel_load_data(train_paths)
df_test  = parallel_load_data(test_paths)

In [3]:
startdate = datetime.datetime(2017,12,1)
df_train['datetime'] = df_train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
df_train['year'] = df_train['datetime'].map(lambda x: x.year)
df_train['month'] = df_train['datetime'].map(lambda x: x.month)
df_train['month'] = df_train['month'].map(lambda x: 5 if x==6 else x)

df_train['DT-M'] = df_train[['year', 'month']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)

In [7]:
tmp_train = df_train[df_train['DT-M']!='2017-12']
dec2017_train = df_train[df_train['DT-M']=='2017-12']
tmp_train['DT-M'].value_counts()

2018-3    101968
2018-1     92510
2018-5     92427
2018-2     85725
2018-4     83571
Name: DT-M, dtype: int64

In [25]:
# n_splits = 5
# Y = dec2017_train[COLUMN_TARGET]
# kfold201712 = list(StratifiedKFold(n_splits=n_splits).split(dec2017_train, Y))

2018-3    101968
2018-1     92510
2018-5     92427
2018-2     85725
2018-4     83571
Name: DT-M, dtype: int64

In [53]:
from sklearn.model_selection import StratifiedKFold, GroupKFold

COLUMN_GROUP = 'DT-M'
n_splits = 5

Y = df_train[COLUMN_TARGET]
Y_kfold = tmp_train[COLUMN_TARGET]
kfold = list(GroupKFold(n_splits=n_splits).split(tmp_train, Y_kfold, tmp_train["DT-M"]))
org_kfold = []

for (trn2018_idx, val2018_idx), (trn2017_idx, val2017_idx) in zip(kfold, kfold201712):
    
    x_train2018 = tmp_train.iloc[trn2018_idx]
    x_valid2018 = tmp_train.iloc[val2018_idx]
    
    x_train2017 = dec2017_train.iloc[trn2017_idx]
    x_valid2017 = dec2017_train.iloc[val2017_idx]
    
    x_train = pd.concat([x_train2018, x_train2017])
    x_valid = pd.concat([x_valid2018, x_valid2017])
    
    display(x_train[COLUMN_GROUP].value_counts())
    display(x_valid[COLUMN_GROUP].value_counts())
    print(x_train[COLUMN_GROUP].shape)
    print(x_valid[COLUMN_GROUP].shape)
    
    trn_idx = x_train.index
    val_idx = x_valid.index
    print(trn_idx.shape, val_idx.shape)
    print(trn_idx)
    org_kfold.append([trn_idx, val_idx])

(461703,) (128837,)
Int64Index([134339, 134340, 134341, 134342, 134343, 134344, 134345, 134346,
            134347, 134348,
            ...
            134329, 134330, 134331, 134332, 134333, 134334, 134335, 134336,
            134337, 134338],
           dtype='int64', length=461703)


SystemExit: 

In [49]:
# to_pkl_gzip(path='../input/0908_ieee__original_KFold_split201712_idx-ID', obj=org_kfold)
kfold = read_pkl_gzip(path='../input/0908_ieee__original_KFold_split201712_idx-ID.gz')